In [1]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2026-01-03 07:26:41--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.04s   

2026-01-03 07:26:41 (29.9 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [2]:
import torch
import torch.nn as nn
from torch.nn import functional as F
#參數
torch.manual_seed(123)
batch_size=64
block_size=256
learning_rate=3e-4
max_iters=5000
eval_iters=200
eval_interval=500
n_embd=384

n_head=6
dropout=0.2
n_layer=6
device='cuda' if torch.cuda.is_available() else 'cpu'

In [3]:
with open ('input.txt','r',encoding='utf-8') as f:
    text = f.read()

chars=sorted(list(set(text)))
vocab_size=len(chars)

#mapping from characters to integers
stoi={ch:i for i,ch in enumerate(chars)} #string to index
itos={i:ch for i,ch in enumerate(chars)} #index to string
encode= lambda s: [stoi[c] for c in s] #string -> list of integers
decode= lambda l: ''.join(itos[i] for i in l) #list of integers -> string

data = torch.tensor(encode(text),dtype=torch.long)

#切訓練與驗證集 9:1
n=int(0.9*len(data))
train_data=data[:n]
valid_data=data[n:]
# data loading
def get_batch(split):
  data=train_data if split=='train' else valid_data
  ix=torch.randint(len(data)-block_size,(batch_size,))  # 隨機生成 batch_size 個合法起始索引不會超出序列範圍
  x=torch.stack([data[i:i+block_size]  for i in ix])
  y=torch.stack([data[i+1:i+block_size+1] for i in ix])
  x,y=x.to(device),y.to(device)
  return x,y

@torch.no_grad() #關閉梯度計算，不需要backprop
def estimate_loss(): #計算eval_iters個隨機batch的平均Loss
  out={}
  model.eval() #關閉drop out與layerNormal等
  for split in ['train','val']:
    losses=torch.zeros(eval_iters)
    for k in range(eval_iters):
      X,Y=get_batch(split)
      logits,loss=model(X,Y)
      losses[k]=loss.item()
    out[split]=losses.mean()
  model.train()
  return out

In [4]:
from torch._C import dtype


class Head(nn.Module):
  def __init__(self,head_size):
    super().__init__()
    self.key=nn.Linear(n_embd,head_size,bias=False)
    self.query=nn.Linear(n_embd,head_size,bias=False) #Linear的公式是 y=x*W^T+b 然後inpute=n_embd是x的每一個 (b, t) 的最後一維向量
    self.value=nn.Linear(n_embd,head_size,bias=False)
    self.register_buffer('tril',torch.tril(torch.ones(block_size,block_size)))

    self.dropout=nn.Dropout(dropout)

  def forward(self,x):
    B,T,C=x.shape
    k=self.key(x) #(B,T,head_size)
    q=self.query(x) #(B,T,head_size)
    #計算Attention score
    wei=q@k.transpose(-2,-1)*(q.size(-1) ** -0.5) #(B,T,H)@(B,H,T) -->(B,T,T)
    wei=wei.masked_fill(self.tril[:T,:T] == 0,float('-inf'))
    wei=F.softmax(wei,dim=-1)
    wei=self.dropout(wei)
    v=self.value(x) #(B,T,head_size)
    out=wei@v #(B,T,T)@(B,T,head_size)-->(B,T,head_size)
    return out


class MutiHeadAttention(nn.Module):
  def __init__(self,num_heads,head_size):
    super().__init__()
    self.head=nn.ModuleList([Head(head_size) for _ in range(num_heads)])
    self.proj=nn.Linear(n_embd,n_embd)
    self.dropout=nn.Dropout(dropout)
  def forward(self,x):
    out=torch.cat([h(x) for h in self.head],dim=-1)
    out=self.dropout(out)
    out=self.proj(out) #為了讓concat後將多頭訊息整合
    return out



class feedforward(nn.Module):
  def __init__(self,n_embd):
    super().__init__()
    self.net=nn.Sequential(
        nn.Linear(n_embd,4*n_embd),
        nn.ReLU(),
        nn.Linear(4*n_embd,n_embd),#將中間非線性輸出壓縮回 n_embd，方便殘差連接
        nn.Dropout(dropout),
    )
  def forward(self,x):
    return self.net(x)

class block(nn.Module):
  def __init__(self,n_embd,n_head):
    super().__init__()
    head_size=n_embd//n_head  #n_embd:embedding dimension, n_head:number of heads we wants
    self.sa=MutiHeadAttention(n_head,head_size)
    self.ffwd=feedforward(n_embd)
    self.ln1=nn.LayerNorm(n_embd)
    self.ln2=nn.LayerNorm(n_embd)

  def forward(self,x):
    x=x+self.sa(self.ln1(x)) #residual connection 跟attention is all you need不同，layerNorm先做
    x=x+self.ffwd(self.ln2(x))
    return x


class BigramlanguangeModel(nn.Module):
  def __init__(self,vocab_size):
    super().__init__() #初始化nn.Module
    self.token_embedding_table=nn.Embedding(vocab_size,n_embd)
    self.position_embedding_table=nn.Embedding(block_size,n_embd)
    self.blocks=nn.Sequential(*[block(n_embd,n_head=n_head) for _ in range(n_layer)] )
    self.ln_f=nn.LayerNorm(n_embd)
    self.lm_head=nn.Linear(n_embd,vocab_size) #decode

  def forward(self,idx,targets=None):
    B,T=idx.shape
    token_emb=self.token_embedding_table(idx) #(B,T,C)
    pos_emb=self.position_embedding_table(torch.arange(T,device=device)) #(T,C)
    x=token_emb+pos_emb #(B,T,C)
    x=self.blocks(x) #(B,T,C)
    logits=self.lm_head(x) #decode (B,T,vocab_size)


    if targets is None:
      loss=None
    else:
      B,T,C=logits.shape
      logits=logits.view(B*T,C) #為了滿足cross_entropy輸入(N,C)輸出(N)其中C是 vocab_size
      targets=targets.view(B*T)
      loss=F.cross_entropy(logits,targets)

    return logits,loss

  def generate(self,idx,max_new_tokens):
    for _ in range(max_new_tokens):
      idx_cond=idx[:, -block_size:] # 因為position_embedding最多看block_size來給分數，所以生成是要有這個限制
      logits,loss=self(idx_cond) #seld() PyTorch會自動呼叫forward
      logits=logits[:,-1,:] #(BxC)取目前每個序列最後一個 token 的預測分數
      prob=F.softmax(logits,dim=-1) #對最後一個維度做solfmax
      idx_next=torch.multinomial(prob,num_samples=1) #(B,1)
      idx=torch.cat((idx,idx_next),dim=1)#(B,T+1)
    return idx








model=BigramlanguangeModel(vocab_size)
m=model.to(device)

#creat a optimizer
optimizer=torch.optim.AdamW(m.parameters(),learning_rate)

#訓練
for iter in range(max_iters):
  if iter%eval_interval==0: #訓練eval_interval次評估一次模型表現
    losses=estimate_loss()
    print(f'step {iter}: train loss {losses["train"]:.4f}, val loss {losses["val"]:.4f}')

  xb,yb=get_batch('train')
  logits,loss=model(xb,yb)
  optimizer.zero_grad(set_to_none=True) #zeroing out all the gradient from the previous step
  loss.backward() #計算參數梯度
  optimizer.step() #用梯度更新參數

contex=torch.zeros((1,1),dtype=torch.long,device=device)
print(decode(m.generate(contex,max_new_tokens=500)[0].tolist()))

step 0: train loss 4.6101, val loss 4.6128
step 500: train loss 1.9416, val loss 2.0371
step 1000: train loss 1.5745, val loss 1.7480
step 1500: train loss 1.4222, val loss 1.6271
step 2000: train loss 1.3279, val loss 1.5574
step 2500: train loss 1.2657, val loss 1.5169
step 3000: train loss 1.2161, val loss 1.4991
step 3500: train loss 1.1695, val loss 1.4808
step 4000: train loss 1.1285, val loss 1.4860
step 4500: train loss 1.0862, val loss 1.4832

To this become scene. Ladies, forget think I met
For that have slain wister you: the glief, sirs
'Tis better your wife be have welching
The irone.
Lady:'' swallowing word, that all sawings
Main all-seed from him shall swears to ye,
Be further will pronounced to you. What the keep hat,
That long kill'd find of contempts: do you hear
A solething a ladim; be acces her found to night
Be grace of the king's hand: Edward sistake,
And he will be hang'd for married his fellock doubt:
And sits Grandage a
